# PS Prediction III: Financial Econometrics and Machine Learning

# Question 1: Data Cleaning and Packages

In [ ]:
pip install wrds

In [1]:
pip install tensorflow

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB 7.9 MB/s eta 0:00:01
   ---------- ----------------------------- 0.7/2.7 MB 9.2 MB/s eta 0:00:01
   ------------------- -------------------- 1.4/2.7 MB 10.7 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 12.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/226.7 kB ? eta -:--:--
   --------------------------------------- 226.7/226.7 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install --upgrade scipy numba daal4py

In [2]:
import pandas as pd
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import scipy
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm, uniform, skew, kurtosis
from sklearn.metrics import r2_score
import statsmodels.api as sm
import os
from arch.univariate import ConstantMean, GARCH, Normal
from arch.__future__ import reindexing
from arch.univariate import arch_model
from arch import arch_model
from arch.univariate import GARCH
from arch.__future__ import reindexing
import yfinance as yf
import wrds
import tensorflow

In [3]:
from numpy import mean
from numpy import std

In [4]:
#item = pd.read_csv("datashare.csv")

In [5]:
#items_permno = pd.Series(item['permno'].unique()).sort_values(ascending=True).tolist()

In [6]:
#item['DATE'] = pd.to_datetime(item['DATE'], format='%Y%m%d', errors='coerce')
#item['DATE'] = item['DATE'].dt.strftime('%Y%m%d')
#print("date range:", item['DATE'].min(), "to", item['DATE'].max())

date range: 19570131 to 20211231


get prices of all securities in datashare from Kelly et Al. through the wrds package by mapping wrds stock returns to our item through the "permno" stock identifier

In [ ]:
db = wrds.Connection(wrds_username='urkesh')

In [ ]:
permno_string = ', '.join(str(permno) for permno in items_permno)
monthly_return_query = f"""
SELECT permno, date, ret AS monthly_return
FROM crsp.msf
WHERE permno IN ({permno_string})
AND date >= '1957-01-31' AND date <= '2021-12-31';
"""
monthly_return_data = db.raw_sql(monthly_return_query)

In [ ]:
#merge the datasets
data = item.merge(monthly_return_data, how='left', left_on=['permno', 'DATE'], right_on=['permno', 'date'])

In [33]:
data = pd.read_csv(r'C:\FinancialEcon Machine Learning\data.csv')

In [34]:
data.drop("date", axis=1, inplace=True)

In [38]:
data['date'] = pd.to_datetime(data['DATE'], format='%Y%m%d', errors='coerce')

data['permno'] = pd.to_numeric(data['permno'], downcast='integer', errors='coerce')

data['permno'] = data['permno'].astype('int64')

print(data['permno'].dtype, data['date'].dtype)

int64 datetime64[ns]


In [39]:
data

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,monthly_return,date
0,10006,19570131,8.224900e+04,1.122846,1.260784,0.047180,9.569953,0.025742,0.046433,0.044843,...,0.013234,9.411565e-08,0.015453,0.008058,0.355638,0.460420,1.120996e-07,37.0,0.064378,1957-01-31
1,10014,19570131,3.903375e+03,0.426734,0.182102,-0.275641,6.237836,0.072103,0.046433,-0.086957,...,0.033305,6.610609e-06,0.047619,0.033495,1.152126,1.169610,9.229146e-08,NaN,0.095238,1957-01-31
2,10022,19570131,9.273250e+03,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046433,-0.060377,...,0.016023,2.286832e-06,0.020833,0.015589,0.815777,0.679803,1.181757e-07,NaN,0.102041,1957-01-31
3,10030,19570131,5.446588e+04,0.926038,0.857547,0.018171,9.825337,0.021700,0.046433,0.044633,...,0.015295,1.464273e-07,0.039326,0.015849,0.739302,1.333656,6.126699e-08,NaN,-0.047091,1957-01-31
4,10057,19570131,4.025000e+04,1.247748,1.556875,0.025785,7.901007,0.025506,0.046433,0.086667,...,0.005954,1.380375e-06,0.056856,0.019945,0.755510,0.410391,3.315790e+00,NaN,-0.090062,1957-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4117295,93423,20211231,3.144398e+06,1.923076,3.698221,-0.673385,16.305232,0.062619,0.491455,-0.110868,...,0.043829,3.698841e-10,0.073435,0.029678,0.399725,9.215947,5.031856e-09,79.0,0.164342,2021-12-31
4117296,93426,20211231,4.326610e+05,1.224523,1.499457,-0.061462,12.419552,0.033723,0.346308,0.007040,...,0.025875,1.097624e-08,0.046055,0.019302,0.481331,1.649750,3.023395e-08,36.0,0.081270,2021-12-31
4117297,93427,20211231,4.092710e+06,0.887083,0.786917,-0.080295,14.884834,0.045395,0.346308,0.151667,...,0.029966,8.593845e-10,0.131563,0.032861,0.440057,2.444487,2.224496e-08,36.0,0.071545,2021-12-31
4117298,93434,20211231,1.130478e+05,0.512942,0.263110,-0.543760,12.243872,0.088124,0.270005,-0.319347,...,0.069943,1.145228e-07,0.036932,0.035858,0.968425,4.557546,3.155353e-08,1.0,-0.065069,2021-12-31


## Extracting the required data and leave the rest out

## mom1m, mom12m, ep, beta, bm, idiovol, mvel1, operprof, roaq, zerotrade, turn, invest, permno, date,  monthly_return

In [40]:
filtered_data = data[['permno', 'date', 'monthly_return', 'mom1m', 'mom12m', 'ep', 'beta', 'bm',
                      'idiovol', 'mvel1', 'operprof', 'roaq', 'zerotrade', 'turn', 'invest']]

In [ ]:
filtered_data

In [41]:
filtered_data.isna().sum()

permno                  0
date                    0
monthly_return      20488
mom1m               31738
mom12m             339105
ep                1030000
beta               400564
bm                1074711
idiovol            400564
mvel1                3070
operprof          1395347
roaq              1699393
zerotrade          309813
turn               350960
invest            1420104
dtype: int64

In [42]:
filtered_data['date'] = pd.to_datetime(filtered_data['date'], format='%Y%m%d')
filtered_data_cols = list(filtered_data.columns.values)
filtered_data['year_month'] = filtered_data['date'].dt.to_period('M')
grouped = filtered_data.groupby([filtered_data['date'].dt.month, filtered_data['date'].dt.year,'permno'])

for col in filtered_data_cols[3:]:
    filtered_data[col] = filtered_data.groupby('year_month')[col].transform(lambda x: x.fillna(x.median()))

C:\Users\urkes\AppData\Local\Temp\ipykernel_41792\2536602564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['date'] = pd.to_datetime(filtered_data['date'], format='%Y%m%d')
C:\Users\urkes\AppData\Local\Temp\ipykernel_41792\2536602564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['year_month'] = filtered_data['date'].dt.to_period('M')
C:\Users\urkes\AppData\Local\Temp\ipykernel_41792\2536602564.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [13]:
filtered_data.isna().sum()

permno                 0
date                   0
monthly_return     20488
mom1m                  0
mom12m                 0
ep                 31991
beta                   0
bm                 72077
idiovol                0
mvel1                  0
operprof           83423
roaq              329143
zerotrade              0
turn                   0
invest             51706
year_month             0
dtype: int64

In [43]:
del filtered_data['year_month']

In [47]:
filtered_data['monthly_return'] = filtered_data['monthly_return'].shift(+1)

C:\Users\urkes\AppData\Local\Temp\ipykernel_41792\2883897418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['monthly_return'] = filtered_data['monthly_return'].shift(+1)


In [49]:
#shifted monthly return
filtered_data = filtered_data.sort_values(by=['permno', 'date'])
filtered_data['shifted_return'] = filtered_data.groupby('permno')['monthly_return'].shift(-1)
filtered_data = filtered_data.groupby('permno').apply(lambda group: group.iloc[1:-1]).reset_index(drop=True)
filtered_data = filtered_data.sort_values(by=['permno', 'date'])

In [50]:
filtered_data = filtered_data.sort_values(by="date")

## Since the explain variables are strictly limited to those 12 variables, we may have to remove data this time to avoid missing data issue.

# Question 2: Implement AdaBoost and NN2 from Gu et al. (2020) to predict next month excess returns.

# Adaboost

In [59]:
filtered_data = filtered_data.dropna(subset=['shifted_return'] + list(filtered_data.columns[3:]))
common_index = filtered_data.index

In [53]:
#common_index = filtered_data.dropna(subset=['shifted_return'] + list(filtered_data.columns[3:])).index

In [67]:
X = data.iloc[:, 2:-1]
y = filtered_data['shifted_return']

In [71]:
training_start = '1957-01-31'
training_end = '1989-12-31'
validating_start = '1990-01-31'
validating_end = '2004-12-31'
testing_start = '2005-01-31'
testing_end = '2020-12-31'

training_mask = (filtered_data['date'] >= training_start) & (filtered_data['date'] <= training_end)
training_index = filtered_data[training_mask].index.intersection(common_index)
X_train = X.loc[training_index]
y_train = y.loc[training_index]

validating_mask = (filtered_data['date'] >= validating_start) & (filtered_data['date'] <= validating_end)
validating_index = filtered_data[validating_mask].index.intersection(common_index)
X_validate = X.loc[validating_index]
y_validate = y.loc[validating_index]

testing_mask = (filtered_data['date'] >= testing_start) & (filtered_data['date'] <= testing_end)
testing_index = filtered_data[testing_mask].index.intersection(common_index)
X_test = X.loc[testing_index]
y_test = y.loc[testing_index]

## Standardize the predictor variables

In [72]:
X_train.mean()

mvel1             744756.497445
beta                   1.007594
betasq                 1.451807
chmom                  0.002983
dolvol                10.157856
                      ...      
std_dolvol             0.968482
std_turn               3.522099
zerotrade              2.071054
sic2                  46.898443
monthly_return         0.012968
Length: 96, dtype: float64

In [73]:
X_train.std()

mvel1             6.090013e+06
beta              6.563374e-01
betasq            1.721748e+00
chmom             5.209434e-01
dolvol            2.704940e+00
                      ...     
std_dolvol        3.947385e-01
std_turn          7.616800e+00
zerotrade         4.140889e+00
sic2              1.979737e+01
monthly_return    1.749351e-01
Length: 96, dtype: float64

In [74]:
(X_train - X_train.mean()) / X_train.std()

,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,mom12m,...,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,monthly_return
631250,-0.121382,-0.257343,-0.434678,-0.141501,NaN,0.332728,0.169723,-0.000703,-0.335023,-0.311935,...,NaN,1.034417,NaN,-0.293017,-0.237251,NaN,NaN,NaN,0.207177,0.512167
378923,-0.121411,NaN,NaN,NaN,NaN,NaN,-1.161919,-0.789155,NaN,NaN,...,NaN,2.915208,NaN,-0.995224,-0.313645,NaN,NaN,NaN,NaN,-0.410390
941114,-0.120258,1.303956,1.173553,-1.983773,NaN,1.629709,-0.569847,2.282837,-1.452141,-0.164121,...,NaN,-0.066344,NaN,1.655041,1.185106,NaN,NaN,NaN,2.025601,1.098465
1480250,-0.121088,0.107185,-0.168343,0.664888,-1.934393,0.395815,-0.126574,0.288519,0.117893,-0.461645,...,1.452767,0.684538,1.649493,2.047673,1.937074,-0.187137,-0.437383,2.586777,-0.550500,-0.074131
1638100,-0.121067,-0.726943,-0.679777,1.394868,-1.172321,0.053312,0.490212,-0.070706,1.313284,0.057806,...,0.266800,-0.108473,-0.212552,-0.995224,-1.195616,2.589024,-0.384862,3.188126,-1.358688,-0.074131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563609,-0.121218,NaN,NaN,NaN,-0.646065,NaN,-1.125388,-0.951386,-0.973081,NaN,...,NaN,0.301941,-0.130267,0.308875,1.165694,0.251013,0.084772,0.602326,NaN,-0.391709
139779,-0.026447,-0.462882,-0.555534,-0.745852,0.845503,-0.871606,0.237283,0.003161,-0.295894,0.292754,...,NaN,-0.626347,-0.212395,-0.729053,-0.976514,-1.188949,-0.375997,-0.500147,NaN,0.014002
1455863,-0.120070,-0.265197,-0.439684,-0.374975,-2.270466,-0.353853,0.373003,0.667162,0.201641,0.896949,...,1.452767,0.579462,5.635174,0.115022,-0.261574,3.455172,-0.416964,-0.500147,1.570995,0.065294
122279,-0.076851,0.417800,0.111070,0.663382,0.299876,-0.738545,-0.176193,-0.211143,0.683385,0.084797,...,NaN,-0.511946,-0.210228,-0.402452,-0.605954,-1.344830,-0.401906,-0.500147,-0.499988,0.093999


In [75]:
X_train

,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,mom12m,...,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,monthly_return
631250,5538.0000,0.838690,0.703401,-0.070731,NaN,0.073765,0.175957,0.010256,-0.061654,-0.044106,...,NaN,0.148269,NaN,0.051282,0.025576,NaN,NaN,NaN,51.0,0.102564
378923,5363.5000,NaN,NaN,NaN,NaN,NaN,-0.151794,-0.105263,NaN,NaN,...,NaN,0.307875,NaN,0.000000,0.023538,NaN,NaN,NaN,NaN,-0.058824
941114,12384.9375,1.863429,3.472368,-1.030451,NaN,0.121216,-0.006070,0.344828,-0.442308,0.035714,...,NaN,0.054857,NaN,0.193548,0.063535,NaN,NaN,NaN,87.0,0.205128
1480250,7330.0000,1.077943,1.161962,0.349352,4.925440,0.076073,0.103031,0.052632,0.092675,-0.124950,...,6.0,0.118578,5.787460e-05,0.222222,0.083604,0.894612,0.190640,1.278261e+01,36.0,0.000000
1638100,7453.8750,0.530474,0.281403,0.729630,6.986797,0.063543,0.254838,0.000000,0.500000,0.155556,...,4.0,0.051282,0.000000e+00,0.000000,0.000000,1.990470,0.590683,1.527273e+01,20.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563609,6539.0625,NaN,NaN,NaN,8.410290,NaN,-0.142803,-0.129032,-0.279070,NaN,...,NaN,0.086110,2.557525e-06,0.095238,0.063017,1.067567,4.167789,4.565217e+00,NaN,-0.055556
139779,583691.6250,0.703787,0.495317,-0.385564,12.444891,0.029704,0.192585,0.010823,-0.048321,0.282428,...,NaN,0.007335,4.878082e-09,0.019438,0.005847,0.499158,0.658205,5.793432e-08,NaN,0.015418
1455863,13530.0000,0.833536,0.694782,-0.192358,4.016383,0.048646,0.225989,0.108108,0.121212,0.608696,...,6.0,0.109661,1.817543e-04,0.081081,0.024927,2.332372,0.346167,1.726974e-06,78.0,0.024390
122279,276734.5000,1.281812,1.643041,0.348567,10.969002,0.034572,0.090818,-0.020576,0.285364,0.170131,...,NaN,0.017043,7.224012e-08,0.043290,0.015737,0.437626,0.460861,1.391896e-07,37.0,0.029412


## Note that the sample mean and standard deviation used to standardize the predictor variables are from the training set. This ensures that there is no information leakage.

In [76]:
X_train = (X_train - X_train.mean()) / X_train.std()
X_validate = (X_validate - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()

In [77]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RepeatedKFold, KFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeRegressor

In [ ]:
pip uninstall numpy -y

In [ ]:
pip install numpy==1.15.0

In [78]:
import numpy as np

In [79]:
y_validate

2156878   -0.050000
3866706    0.000000
1770748    0.048611
811456     0.130435
1664830    0.058823
             ...   
2886971   -0.135560
2095572    0.003009
1547780   -0.081793
2674393    0.073394
3491323    0.137605
Name: shifted_return, Length: 1359683, dtype: float64

In [80]:
crossvalidation = KFold(n_splits = 5, shuffle = True, random_state = 1)
for depth in range(1, 10):
    tree_regressor = tree.DecisionTreeRegressor(max_depth = depth, random_state = 1)
    if tree_regressor.fit(X_train, y_train).tree_.max_depth < depth:
        break
    score = np.mean(cross_val_score(tree_regressor, X_train, y_train, scoring = 'neg_mean_squared_error',
                                    cv = crossvalidation, n_jobs = 1))
    print(depth, score)

KeyboardInterrupt: 

# Since technically, we are only tuning tree depth to be 1 or 2 and 2 is a little bit better than 1, we will fix tree depth as 2.

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # define number of trees to consider
    n_trees = [100, 200, 300, 400, 500]
    for n in n_trees:
        models[str(n)] = AdaBoostRegressor(n_estimators=n)
    return models
 
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits = 3, shuffle = True, random_state = 1)
    # evaluate the model and collect the results
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=1)
    return scores
 

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_validate, y_validate)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

# We interrupted the kernel because the progress 100 -> 200 -> 300 trees has already shown a trend. It looks like 100 trees is the optimal choice.

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # define number of trees to consider
    learning_rates = [0.01, 0.1]
    for rate in learning_rates:
        models[str(rate)] = AdaBoostRegressor(learning_rate=rate)
    return models
 
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits = 3, shuffle = True, random_state = 1)
    # evaluate the model and collect the results
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=1)
    return scores
 

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_validate, y_validate)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

# For the learning rate, looks like taking smaller step is better. Therefore, we will choose 0.01.

# To summarize our best parameters
## 1. Tree depth: 2
## 2. Number of trees: 100
## 3. Learning rate: 0.01

In [ ]:
# Merge training and validating data to prepare for the final training
X_train = X_train.append(X_validate)
y_train = y_train.append(y_validate)

base_estimator = DecisionTreeRegressor(max_depth=2)
model = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=100, learning_rate=0.01)
model.fit(X_train, y_train)

# Note to teammate: the model is ready. However, using the entire training set will take long, please use a smaller subset to get the final model.

In [93]:
from sklearn.utils import shuffle

X_full_train = pd.concat([X_train, X_validate])
y_full_train = pd.concat([y_train, y_validate])

# Shuffle the data and take a random 25% sample of the full training set
X_sample, y_sample = shuffle(X_full_train, y_full_train, random_state=42)
X_sample = X_sample.sample(frac=0.25, random_state=42)
y_sample = y_sample.loc[X_sample.index]

# Initialize the base estimator and the AdaBoostRegressor
base_estimator = DecisionTreeRegressor(max_depth=2)
model = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=100, learning_rate=0.01)

# Fit the model on the sampled data
model.fit(X_sample, y_sample)

ValueError: Input X contains NaN.
AdaBoostRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [117]:
X_sample, y_sample = shuffle(X_full_train, y_full_train, random_state=42)

sample_frac = 0.25 #25% of data
X_sample = X_sample.sample(frac=sample_frac, random_state=42)
y_sample = y_sample.sample(frac=sample_frac, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_sample_imputed = imputer.fit_transform(X_sample)

model.fit(X_sample_imputed, y_sample)

C:\Users\urkes\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=2),
                  learning_rate=0.01, n_estimators=100)

In [124]:

val_predictions = model.predict(X_sample_imputed) #validate the model
val_mse = mean_squared_error(y_val, val_predictions)
print(f'Validation Mean Squared Error: {val_mse}')


test_predictions = model.predict(X_sample_imputed) # Testing the model
test_mse = mean_squared_error(y_test, test_predictions)
print(f'Test Mean Squared Error: {test_mse}')

NameError: name 'y_val' is not defined

# Neural Network

In [81]:
pip install keras-tuner

   ---------------------------------------- 0.0/128.9 kB ? eta -:--:--
   ------------------- -------------------- 61.4/128.9 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 128.9/128.9 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [82]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch

In [83]:
training_start = '1957-01-31'
training_end = '1989-12-31'
validating_start = '1990-01-31'
validating_end = '2004-12-31'
testing_start = '2005-01-31'
testing_end = '2020-12-31'

training_mask = (filtered_data['date'] >= training_start) & (filtered_data['date'] <= training_end)
training_index = filtered_data[training_mask].index.intersection(common_index)
X_train = X.loc[training_index]
y_train = y.loc[training_index]

validating_mask = (filtered_data['date'] >= validating_start) & (filtered_data['date'] <= validating_end)
validating_index = filtered_data[validating_mask].index.intersection(common_index)
X_validate = X.loc[validating_index]
y_validate = y.loc[validating_index]

testing_mask = (filtered_data['date'] >= testing_start) & (filtered_data['date'] <= testing_end)
testing_index = filtered_data[testing_mask].index.intersection(common_index)
X_test = X.loc[testing_index]
y_test = y.loc[testing_index]

In [84]:
X_train = (X_train - X_train.mean()) / X_train.std()
X_validate = (X_validate - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()

In [100]:
from keras.callbacks import EarlyStopping

modelnn = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(1)  # Assuming a regression problem; change accordingly for classification
])

# Compile the model
modelnn.compile(optimizer=Adam(learning_rate=0.001),  # Start with 0.001
              loss='mean_squared_error')  # Change loss function based on problem

# Early stopping
early_stopping = EarlyStopping(patience=5)

# Train the model
history = modelnn.fit(X_train, y_train, epochs=100, batch_size=10000, validation_data=(X_validate, y_validate), callbacks=[early_stopping])

Epoch 1/100
252/252 [==============================] - 6s 20ms/step - loss: nan - val_loss: nan
Epoch 2/100
252/252 [==============================] - 3s 12ms/step - loss: nan - val_loss: nan
Epoch 3/100
252/252 [==============================] - 3s 12ms/step - loss: nan - val_loss: nan
Epoch 4/100
252/252 [==============================] - 3s 11ms/step - loss: nan - val_loss: nan
Epoch 5/100
252/252 [==============================] - 3s 12ms/step - loss: nan - val_loss: nan


In [101]:
def build_model(hp):
    model = Sequential([
        Dense(32, activation='relu', input_shape=(X.shape[1],)),
        BatchNormalization(),
        Dense(16, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])

    # Tune the learning rate (from 0.001 to 0.01) and shrinkage parameter α (from 10^−5 to 10^−3)
    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01])
    alpha = hp.Float('alpha', min_value=1e-5, max_value=1e-3, sampling='log')

    modelnn.compile(optimizer=Adam(learning_rate=learning_rate, beta_1=1-alpha),
                  loss='mean_squared_error')

    return model

# Create a tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',  
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='hparam_tuning'
)

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Execute the search
tuner.search(X_validate, y_validate, epochs=100, batch_size=10000, validation_data=(X_validate, y_validate), callbacks=[early_stopping])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Reloading Tuner from my_dir\hparam_tuning\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.01              |0.001             |learning_rate
3.0476e-05        |0.00010248        |alpha



Traceback (most recent call last):
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras\src\utils\traceback_uti

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "C:\Users\urkes\anaconda3\lib\site-packages\keras_tuner\src\engine\hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "C:

In [102]:


# Merge training and validating data to prepare for the final trainings
X_train = pd.concat([X_train, X_validate])
y_train = pd.concat([y_train, y_validate])

modelnn = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(1)  #Assuming a regression problem; change accordingly for classification
])

# Compile the model
modelnn.compile(optimizer=Adam(learning_rate=0.001),  # Start with 0.001
              loss='mean_squared_error')  # Change loss function based on problem

# Early stopping
early_stopping = EarlyStopping(patience=5)

# Train the model
modelnn.fit(X_train, y_train, epochs=100, batch_size=10000, callbacks=[early_stopping])

Epoch 1/100
388/388 [==============================] - 5s 10ms/step - loss: nan
Epoch 2/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 3/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 4/100
388/388 [==============================] - 4s 11ms/step - loss: nan
Epoch 5/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 6/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 7/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 8/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 9/100
388/388 [==============================] - 4s 11ms/step - loss: nan
Epoch 10/100
388/388 [==============================] - 4s 11ms/step - loss: nan
Epoch 11/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 12/100
388/388 [==============================] - 4s 10ms/step - loss: nan
Epoch 13/100
388/388 [===============

# Note for teammate: The code for NN is written, but my Python version is not compatible with the tensorflow package. Please use your computer to run this.

#

# Question 3: Show how the models performs on the test set. 

Adaboost model performance

In [105]:
imputer = SimpleImputer(strategy='mean')  # or the strategy you used
X_test_imputed = imputer.fit_transform(X_test)

In [ ]:
# Predict with AdaBoost Model
y_pred_adaboost = model.predict(X_test_imputed)

# Calculate R-squared for AdaBoost
r2_adaboost = r2_score(y_test, y_pred_adaboost)
print(f"R-squared for AdaBoost model: {r2_adaboost}")

# Calculate Sharpe Ratios for AdaBoost
deciles = pd.qcut(y_pred_adaboost, 10, labels=False)
sharpe_ratios_adaboost = [np.mean(y_test[deciles == d]) / np.std(y_test[deciles == d]) for d in range(10)]
print("Sharpe Ratios for each decile (AdaBoost):", sharpe_ratios_adaboost)

NN2 Model Performance

In [118]:
y_pred_nn = modelnn.predict(X_test_imputed).flatten()

# Calculate R-squared for Neural Network
r2_nn = r2_score(y_test, y_pred_nn)
print(f"R-squared for Neural Network model: {r2_nn}")

# Calculate Sharpe Ratios for Neural Network
deciles_nn = pd.qcut(y_pred_nn, 10, labels=False)
sharpe_ratios_nn = [np.mean(y_test[deciles_nn == d]) / np.std(y_test[deciles_nn == d]) for d in range(10)]
print("Sharpe Ratios for each decile (Neural Network):", sharpe_ratios_nn)

35234/35234 [==============================] - 24s 690us/step


ValueError: Input contains NaN.

In [ ]:
# Check for NaN in the imputed test set
if np.isnan(X_test_imputed).any():
    print("NaN values found in X_test_imputed")
else:
    # Predict with Neural Network Model
    y_pred_nn = modelnn.predict(X_test_imputed).flatten()

    # Check for NaN in predictions
    if np.isnan(y_pred_nn).any():
        print("NaN values found in predictions")
    else:
        # Calculate R-squared for Neural Network
        r2_nn = r2_score(y_test, y_pred_nn)
        print(f"R-squared for Neural Network model: {r2_nn}")

        # Calculate Sharpe Ratios for Neural Network
        deciles_nn = pd.qcut(y_pred_nn, 10, labels=False)
        sharpe_ratios_nn = [np.mean(y_test[deciles_nn == d]) / np.std(y_test[deciles_nn == d]) for d in range(10)]
        print("Sharpe Ratios for each decile (Neural Network):", sharpe_ratios_nn)

#

# Outside source of help: ChatGPT, TA Session, Machine Learning Mastery